# Librerias

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost
from sklearn import metrics, svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC 
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from feature_engine.encoding import OneHotEncoder, CountFrequencyEncoder, OrdinalEncoder, RareLabelEncoder
from lightgbm import LGBMClassifier
from sklearn.ensemble import StackingClassifier

# Analisis exploratorio

In [2]:
test_df = pd.read_csv('z_test.csv')

In [3]:
# Leo el DF
df = pd.read_csv('z_train.csv')

In [4]:
# Reviso los campos
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,21651,city_176,0.764,NaN,Has relevent experience,Part time course,Graduate,STEM,11,NaN,NaN,1,24,1.0


In [5]:
# Veo tipo de datos y cantidad de nulos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15326 entries, 0 to 15325
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             15326 non-null  int64  
 1   city                    15326 non-null  object 
 2   city_development_index  15326 non-null  float64
 3   gender                  11725 non-null  object 
 4   relevent_experience     15326 non-null  object 
 5   enrolled_university     15024 non-null  object 
 6   education_level         14957 non-null  object 
 7   major_discipline        13089 non-null  object 
 8   experience              15276 non-null  object 
 9   company_size            10592 non-null  object 
 10  company_type            10435 non-null  object 
 11  last_new_job            14987 non-null  object 
 12  training_hours          15326 non-null  int64  
 13  target                  15326 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [6]:
df.describe()

,enrollee_id,city_development_index,training_hours,target
count,15326.000000,15326.000000,15326.000000,15326.000000
mean,16838.278807,0.829929,65.279068,0.249511
std,9625.018286,0.122668,60.079482,0.432744
min,1.000000,0.448000,1.000000,0.000000
25%,8482.250000,0.743000,23.000000,0.000000
50%,16977.000000,0.910000,47.000000,0.000000
75%,25144.000000,0.920000,89.000000,0.000000
max,33380.000000,0.949000,336.000000,1.000000


In [7]:
df.describe(include='object')

,city,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job
count,15326,11725,15326,15024,14957,13089,15276,10592,10435,14987
unique,123,3,2,3,5,6,22,8,6,6
top,city_103,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,1
freq,3497,10577,11068,11071,9265,11609,2633,2448,7873,6458


In [8]:
# Veo los valores unicos de cada columna categorica
for column in df.select_dtypes('object'):
    print (column)
    print (df[column].unique())
    print('-'*80)

city
['city_103' 'city_40' 'city_21' 'city_115' 'city_176' 'city_160' 'city_46'
 'city_61' 'city_114' 'city_159' 'city_102' 'city_67' 'city_100' 'city_16'
 'city_71' 'city_104' 'city_64' 'city_101' 'city_105' 'city_73' 'city_162'
 'city_75' 'city_41' 'city_11' 'city_93' 'city_90' 'city_13' 'city_36'
 'city_20' 'city_57' 'city_152' 'city_19' 'city_65' 'city_173' 'city_136'
 'city_98' 'city_50' 'city_82' 'city_157' 'city_150' 'city_175' 'city_94'
 'city_28' 'city_83' 'city_59' 'city_165' 'city_145' 'city_142' 'city_138'
 'city_12' 'city_37' 'city_116' 'city_23' 'city_99' 'city_149' 'city_10'
 'city_80' 'city_70' 'city_158' 'city_123' 'city_7' 'city_72' 'city_106'
 'city_78' 'city_45' 'city_109' 'city_24' 'city_43' 'city_89' 'city_134'
 'city_144' 'city_91' 'city_146' 'city_133' 'city_126' 'city_9' 'city_128'
 'city_167' 'city_27' 'city_84' 'city_54' 'city_97' 'city_39' 'city_79'
 'city_76' 'city_74' 'city_81' 'city_131' 'city_44' 'city_117' 'city_155'
 'city_26' 'city_118' 'city_143' 'ci

In [9]:
df['target'].unique()
# 0 no busca y 1 si busca

array([1., 0.])

In [10]:
nulls = df.isnull().mean()
nulls

enrollee_id               0.000000
city                      0.000000
city_development_index    0.000000
gender                    0.234960
relevent_experience       0.000000
enrolled_university       0.019705
education_level           0.024077
major_discipline          0.145961
experience                0.003262
company_size              0.308887
company_type              0.319131
last_new_job              0.022119
training_hours            0.000000
target                    0.000000
dtype: float64

In [11]:
# hago una lista de el numero de nulos que tengo en cada columna
nulls = df.isnull().mean()
most_nulls = [col_name for col_name in nulls.index if nulls[col_name] >.1]

In [12]:
# Veo la lista de columnas con mas d 10% d nulos
most_nulls

['gender', 'major_discipline', 'company_size', 'company_type']

In [13]:
# creo una funcion para ver cuantos valores hay de cada etiqueta en las columnas con muchos nulos
def category_pct (column):
    for value in df[column].unique():
        if str(value) != 'nan' :
            print(value)
            print(df[df[column]== value]['target'].value_counts(normalize = True))
            print('*'*50)
        else:
            print(value)
            print(df[column].isnull().sum())
            print('*'*50)

In [14]:
# Reviso los valores para todas las columnas
for col in most_nulls:
    category_pct(col)

Male
0.0    0.771958
1.0    0.228042
Name: target, dtype: float64
**************************************************
nan
3601
**************************************************
Female
0.0    0.742485
1.0    0.257515
Name: target, dtype: float64
**************************************************
Other
0.0    0.753333
1.0    0.246667
Name: target, dtype: float64
**************************************************
STEM
0.0    0.738909
1.0    0.261091
Name: target, dtype: float64
**************************************************
Business Degree
0.0    0.725869
1.0    0.274131
Name: target, dtype: float64
**************************************************
nan
2237
**************************************************
Humanities
0.0    0.788679
1.0    0.211321
Name: target, dtype: float64
**************************************************
Arts
0.0    0.803922
1.0    0.196078
Name: target, dtype: float64
**************************************************
No Major
0.0    0.748603
1.0    0.251397


In [15]:
# Veo que la distribucion de estos valores no afecta la probabilidad de querer o no buscar un nuevo trabajo
# Con esto concluyo que puedo eliminar estas columnas con gran cantidad de nulos
clean_df = df.drop(most_nulls, axis = 1)

In [16]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15326 entries, 0 to 15325
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             15326 non-null  int64  
 1   city                    15326 non-null  object 
 2   city_development_index  15326 non-null  float64
 3   relevent_experience     15326 non-null  object 
 4   enrolled_university     15024 non-null  object 
 5   education_level         14957 non-null  object 
 6   experience              15276 non-null  object 
 7   last_new_job            14987 non-null  object 
 8   training_hours          15326 non-null  int64  
 9   target                  15326 non-null  float64
dtypes: float64(2), int64(2), object(6)
memory usage: 1.2+ MB


In [17]:
nulls = clean_df.isnull().mean().sort_values(ascending = False)
nulls = nulls[nulls.values > 0]
nulls

education_level        0.024077
last_new_job           0.022119
enrolled_university    0.019705
experience             0.003262
dtype: float64

In [18]:
clean_df = clean_df.drop('enrollee_id', axis = 1)

In [19]:
for col in clean_df.columns:
    moda = str(clean_df[col].mode()[0])
    clean_df[col].fillna(value=moda, inplace = True)

In [20]:
clean_df.isnull().mean()

city                      0.0
city_development_index    0.0
relevent_experience       0.0
enrolled_university       0.0
education_level           0.0
experience                0.0
last_new_job              0.0
training_hours            0.0
target                    0.0
dtype: float64

# Cleaning

In [21]:
def cleaning(df):
    
    clean_df = df.drop('enrollee_id', axis = 1)
    
    clean_df['experience'].replace('>20','21', inplace = True)
    clean_df['experience'].replace('<1','0', inplace = True)
    
    clean_df['last_new_job'].replace('>4','5', inplace = True)
    clean_df['last_new_job'].replace('never','0', inplace = True)
    
    clean_df[['experience','last_new_job']] = clean_df[['experience','last_new_job']].apply(pd.to_numeric)
    
    for col in ['experience','last_new_job']:
        mediana = clean_df[col].median()
        clean_df[col].fillna(value = mediana, inplace = True)

    for col in clean_df.columns:
        moda = str(clean_df[col].mode()[0])
        clean_df[col].fillna(value='no_info', inplace = True)
    
    return clean_df

In [22]:
def norm_std(df):
    
    columns = [col for col in df.select_dtypes('number').columns if col != 'target']

    scaler = MinMaxScaler()
    df[columns]=scaler.fit_transform(df[columns])
    
    return df

In [23]:
def mean_encoding(df):
    enc_freq = CountFrequencyEncoder(encoding_method='frequency')
    enc_freq.fit(df)
    city_dict = enc_freq.encoder_dict_['city']
    df['city'] =  df['city'].map(city_dict) 
    return df

In [24]:
def dummies(df):
    df_dummies = pd.get_dummies(df.select_dtypes('object'), drop_first = True)
    return pd.concat([df.select_dtypes('number'),df_dummies], axis=1)

In [25]:
def data_transformation(df):
    df = cleaning(df)
    df = norm_std(df)
    #df = mean_encoding(df)
    df = dummies(df)
    return(df)

In [26]:
df_2 = data_transformation(df)
df_2.columns = [col.replace('<', '') for col in df_2.columns]
df_2.columns = [col.replace(']', '') for col in df_2.columns]
df_2.head(2)

,city_development_index,experience,last_new_job,training_hours,target,city_city_10,city_city_100,city_city_101,city_city_102,city_city_103,...,company_size_500-999,company_size_5000-9999,company_size_10,company_size_no_info,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,company_type_no_info
0,0.942116,1.000000,0.2,0.104478,1.0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
1,0.654691,0.714286,1.0,0.137313,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


# Smote

## Feat. Fran Galán

In [27]:
x = df_2.drop('target', axis = 1)
y = df_2['target']
oversample = SMOTE()
x, y = oversample.fit_resample(x,y)
df_smote = pd.concat([x,y], axis = 1)
df_smote

,city_development_index,experience,last_new_job,training_hours,city_city_10,city_city_100,city_city_101,city_city_102,city_city_103,city_city_104,...,company_size_5000-9999,company_size_10,company_size_no_info,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,company_type_no_info,target
0,0.942116,1.000000,0.200000,0.104478,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,1.0
1,0.654691,0.714286,1.000000,0.137313,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0.0
2,0.351297,0.238095,0.000000,0.244776,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0.0
3,0.680639,0.000000,0.000000,0.152239,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1.0
4,0.630739,0.523810,0.200000,0.068657,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22999,0.942116,0.224598,0.188662,0.058239,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,1.0
23000,0.942116,0.984580,0.632383,0.058820,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,1.0
23001,0.157685,0.346955,0.038088,0.083014,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1.0
23002,0.351297,0.376574,0.381611,0.468916,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1.0


# Models

## LGBM, XGBOOST, SVM, K-NEIGHBORS

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, stratify = y, random_state = 1)

In [32]:
model = LGBMClassifier()

parameters = {
    'learning_rate': [0.01, 0.1, 0.001],
    'max_depth':[3, 4, 5],
    'n_estimators': [10, 50, 100]}
# 
gs = GridSearchCV(model, parameters)
gs.fit(x_train, y_train, verbose=False)
print(gs.best_params_)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^m

In [34]:
model = svm.SVC()

parameters = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']} 
# 
gs = GridSearchCV(model, parameters)
gs.fit(x_train, y_train)
print(gs.best_params_)

{'C': 1, 'gamma': 1, 'kernel': 'rbf'}


In [38]:
model = xgboost.XGBClassifier()

parameters = {
    'learning_rate': [0.01, 0.1, 0.001],
    'max_depth':[3, 4, 5],
    'objective': ['binary:logistic']}
# 
gs = GridSearchCV(model, parameters)
gs.fit(x_train, y_train, verbose=False)
print(gs.best_params_)

{'learning_rate': 0.1, 'max_depth': 5, 'objective': 'binary:logistic'}


In [40]:
model = KNeighborsClassifier()

parameters = {
    'n_neighbors': [3,4,5,6],
    'weights':['uniform', 'distance'],
    'metric' : ['euclidean','manhattan']}
# 
gs = GridSearchCV(model, parameters)
gs.fit(x_train, y_train)
print(gs.best_params_)

{'metric': 'manhattan', 'n_neighbors': 4, 'weights': 'distance'}


In [29]:
# Usar best params
models = [xgboost.XGBClassifier(learning_rate = 0.1, max_depth = 5, objective = 'binary:logistic'), 
          svm.SVC(C = 1, gamma = 1, kernel = 'rbf'),
          LGBMClassifier(learning_rate = 0.1, max_depth = 5, n_estimators = 100),
         KNeighborsClassifier(n_neighbors=4, weights = 'distance', metric = 'manhattan')]

In [30]:
model_label = ['xbc', 'SVC', 'LGBM', 'KNN']

# Submission df

In [31]:
cols = [col for col in df_2.columns if col not in test_df.columns and col != 'target']

In [32]:
def zero_cols (df_train, df_test):
    cols = [col for col in df_train.columns if col not in df_test.columns and col != 'target']
    for col in cols:
        df_test[col] = 0
    df_test = df_test[df_train.drop('target', axis = 1).columns]
    return df_test

In [33]:
def mean_encoding_sub(df, df_train):
    df_train = cleaning(df_train)
    enc_freq = CountFrequencyEncoder(encoding_method='frequency')
    enc_freq.fit(df_train)
    city_dict = enc_freq.encoder_dict_['city']
    df['city'] =  df['city'].map(city_dict) 
    return df

In [35]:
def test_data_transformation(df, df_train, df_original):
    df = cleaning(df)
    df = norm_std(df)
    df = dummies(df)
    df = zero_cols(df_train, df)
    return(df)

In [36]:
sub_df = test_data_transformation(test_df, df_2, df)
sub_df.head(2)

,city_development_index,experience,last_new_job,training_hours,city_city_10,city_city_100,city_city_101,city_city_102,city_city_103,city_city_104,...,company_size_500-999,company_size_5000-9999,company_size_10,company_size_no_info,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,company_type_no_info
0,0.942116,0.238095,0.2,0.229851,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,0.882236,0.285714,0.2,0.104478,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [37]:
sub_df.columns = [col.replace('<', '') for col in sub_df.columns]
sub_df.columns = [col.replace(']', '') for col in sub_df.columns]


In [58]:
for i, model in enumerate(models):
    md = model
    md.fit(x_train, y_train)
    y_pred = md.predict(x_test)
    md.fit(x,y)
    y_pred_full = md.predict(x)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    accuracy_full = metrics.accuracy_score(y, y_pred_full)
    print('-'*50)
    print(model_label[i])
    print(f'Accuracy score: {accuracy}')
    print(confusion_matrix(y_test, y_pred))
    print('.'*50)
    print(f'Full accuracy score: {accuracy_full}')
    print(confusion_matrix(y, y_pred_full))
    
    first_preds = md.predict(sub_df)
    first_preds_df = sub_df.copy()
    first_preds_df['target'] = first_preds
    
    second_train_df = pd.concat([df_smote,first_preds_df], axis = 0)
    
    print(second_train_df.shape)
    
    x_est = second_train_df.drop('target', axis = 1)
    y_est = second_train_df['target']
    md.fit(x_est,y_est)
    y_pred_est = md.predict(x_est)
    accuracy_est = metrics.accuracy_score(y_est, y_pred_est)
    print(f'Est accuracy score: {accuracy_est}')
    print(confusion_matrix(y_est, y_pred_est))    
    
    sub_pred = md.predict(sub_df)
    submission = pd.DataFrame(test_df['enrollee_id'])
    submission['target'] = sub_pred
    print(submission['target'].value_counts())
    print(submission.shape)
    submission.to_csv(f'{model_label[i]}final.csv', index = False)
    

--------------------------------------------------
xbc
Accuracy score: 0.844599000217344
[[1928  372]
 [ 343 1958]]
..................................................
Full accuracy score: 0.8485046078942793
[[9656 1846]
 [1639 9863]]
(26836, 159)
Est accuracy score: 0.8660381576986138
[[12281  1867]
 [ 1728 10960]]
0.0    2640
1.0    1192
Name: target, dtype: int64
(3832, 2)
--------------------------------------------------
SVC
Accuracy score: 0.8393827428819822
[[1958  342]
 [ 397 1904]]
..................................................
Full accuracy score: 0.9051904016692749
[[10215  1287]
 [  894 10608]]
(26836, 159)
Est accuracy score: 0.9160828737516769
[[12969  1302]
 [  950 11615]]
0.0    2770
1.0    1062
Name: target, dtype: int64
(3832, 2)
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
--------------------------------------------------
LGBM
Accuracy score: 0.8406868072158227
[[1921  379]
 [ 35

# Stacking!

In [65]:
estimators_m = [('LGBM', LGBMClassifier(learning_rate = 0.1, max_depth = 5, n_estimators = 100))
                ,('SVC',svm.SVC(C = 1, gamma = 1,kernel = 'rbf'))]


md = StackingClassifier(estimators=estimators_m, final_estimator=LogisticRegression())

In [66]:
md.fit(x_train, y_train)
y_pred = md.predict(x_test)
md.fit(x,y)
y_pred_full = md.predict(x)
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy_full = metrics.accuracy_score(y, y_pred_full)
print('-'*50)
print('lgbsvc')
print(f'Accuracy score: {accuracy}')
print(confusion_matrix(y_test, y_pred))
print('.'*50)
print(f'Full accuracy score: {accuracy_full}')
print(confusion_matrix(y, y_pred_full))

first_preds = md.predict(sub_df)
first_preds_df = sub_df.copy()
first_preds_df['target'] = first_preds

second_train_df = pd.concat([df_smote,first_preds_df], axis = 0)

print(second_train_df.shape)

x_est = second_train_df.drop('target', axis = 1)
y_est = second_train_df['target']
md.fit(x_est,y_est)
y_pred_est = md.predict(x_est)
accuracy_est = metrics.accuracy_score(y_est, y_pred_est)
print(f'Est accuracy score: {accuracy_est}')
print(confusion_matrix(y_est, y_pred_est))    

sub_pred = md.predict(sub_df)
submission = pd.DataFrame(test_df['enrollee_id'])
submission['target'] = sub_pred
print(submission['target'].value_counts())
print(submission.shape)
submission.to_csv(f'lgbsvcfinal.csv', index = False)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^m

In [67]:
estimators_m = [('KNN', KNeighborsClassifier(n_neighbors=4, weights = 'distance', metric = 'manhattan'))
                ,('LGBM',LGBMClassifier(learning_rate = 0.1, max_depth = 5, n_estimators = 100))]


md  = StackingClassifier(estimators=estimators_m, final_estimator=LogisticRegression())

In [68]:
md.fit(x_train, y_train)
y_pred = md.predict(x_test)
md.fit(x,y)
y_pred_full = md.predict(x)
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy_full = metrics.accuracy_score(y, y_pred_full)
print('-'*50)
print('knnlgbm')
print(f'Accuracy score: {accuracy}')
print(confusion_matrix(y_test, y_pred))
print('.'*50)
print(f'Full accuracy score: {accuracy_full}')
print(confusion_matrix(y, y_pred_full))

first_preds = md.predict(sub_df)
first_preds_df = sub_df.copy()
first_preds_df['target'] = first_preds

second_train_df = pd.concat([df_smote,first_preds_df], axis = 0)

print(second_train_df.shape)

x_est = second_train_df.drop('target', axis = 1)
y_est = second_train_df['target']
md.fit(x_est,y_est)
y_pred_est = md.predict(x_est)
accuracy_est = metrics.accuracy_score(y_est, y_pred_est)
print(f'Est accuracy score: {accuracy_est}')
print(confusion_matrix(y_est, y_pred_est))    

sub_pred = md.predict(sub_df)
submission = pd.DataFrame(test_df['enrollee_id'])
submission['target'] = sub_pred
print(submission['target'].value_counts())
print(submission.shape)
submission.to_csv(f'knnlgbmfinal.csv', index = False)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^m

In [69]:
estimators_m = [('KNN', KNeighborsClassifier(n_neighbors=4, weights = 'distance', metric = 'manhattan'))
                ,('SVC',svm.SVC(C = 1, gamma = 1, kernel = 'rbf'))]


md  = StackingClassifier(estimators=estimators_m, final_estimator=LogisticRegression())

In [70]:
md.fit(x_train, y_train)
y_pred = md.predict(x_test)
md.fit(x,y)
y_pred_full = md.predict(x)
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy_full = metrics.accuracy_score(y, y_pred_full)
print('-'*50)
print('knnsvc')
print(f'Accuracy score: {accuracy}')
print(confusion_matrix(y_test, y_pred))
print('.'*50)
print(f'Full accuracy score: {accuracy_full}')
print(confusion_matrix(y, y_pred_full))

first_preds = md.predict(sub_df)
first_preds_df = sub_df.copy()
first_preds_df['target'] = first_preds

second_train_df = pd.concat([df_smote,first_preds_df], axis = 0)

print(second_train_df.shape)

x_est = second_train_df.drop('target', axis = 1)
y_est = second_train_df['target']
md.fit(x_est,y_est)
y_pred_est = md.predict(x_est)
accuracy_est = metrics.accuracy_score(y_est, y_pred_est)
print(f'Est accuracy score: {accuracy_est}')
print(confusion_matrix(y_est, y_pred_est))    

sub_pred = md.predict(sub_df)
submission = pd.DataFrame(test_df['enrollee_id'])
submission['target'] = sub_pred
print(submission['target'].value_counts())
print(submission.shape)
submission.to_csv(f'knnsvcfinal.csv', index = False)

--------------------------------------------------
knnsvc
Accuracy score: 0.8391653988263421
[[1928  372]
 [ 368 1933]]
..................................................
Full accuracy score: 0.9652669101025908
[[11332   170]
 [  629 10873]]
(26836, 159)
Est accuracy score: 0.9681770755701297
[[14023   143]
 [  711 11959]]
0.0    2664
1.0    1168
Name: target, dtype: int64
(3832, 2)


In [71]:
estimators_m = [('KNN', KNeighborsClassifier(n_neighbors=4, weights = 'distance', metric = 'manhattan'))
                ,('SVC',svm.SVC(C = 1, gamma = 1, kernel = 'rbf'))
               ,('LGBM',LGBMClassifier(learning_rate = 0.1, max_depth = 5, n_estimators = 100))]


md  = StackingClassifier(estimators=estimators_m, final_estimator=LogisticRegression())

In [ ]:
md.fit(x_train, y_train)
y_pred = md.predict(x_test)
md.fit(x,y)
y_pred_full = md.predict(x)
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy_full = metrics.accuracy_score(y, y_pred_full)
print('-'*50)
print('bazooka')
print(f'Accuracy score: {accuracy}')
print(confusion_matrix(y_test, y_pred))
print('.'*50)
print(f'Full accuracy score: {accuracy_full}')
print(confusion_matrix(y, y_pred_full))

first_preds = md.predict(sub_df)
first_preds_df = sub_df.copy()
first_preds_df['target'] = first_preds

second_train_df = pd.concat([df_smote,first_preds_df], axis = 0)

print(second_train_df.shape)

x_est = second_train_df.drop('target', axis = 1)
y_est = second_train_df['target']
md.fit(x_est,y_est)
y_pred_est = md.predict(x_est)
accuracy_est = metrics.accuracy_score(y_est, y_pred_est)
print(f'Est accuracy score: {accuracy_est}')
print(confusion_matrix(y_est, y_pred_est))    

sub_pred = md.predict(sub_df)
submission = pd.DataFrame(test_df['enrollee_id'])
submission['target'] = sub_pred
print(submission['target'].value_counts())
print(submission.shape)
submission.to_csv(f'bazookafinal.csv', index = False)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [38]:
for i, model in enumerate(models):
    md = model
    md.fit(x_train, y_train)
    yt_pred = md.predict(x_train)
    y_pred = md.predict(x_test)
    md.fit(x,y)
    y_pred_full = md.predict(x)
    tr_accuracy = metrics.accuracy_score(y_train, yt_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    accuracy_full = metrics.accuracy_score(y, y_pred_full)
    print('-'*50)
    print(model_label[i])
    print(f'Train ccuracy score: {tr_accuracy}')
    print(f'Test ccuracy score: {accuracy}')
    print(confusion_matrix(y_test, y_pred))
    print('.'*50)
    print(f'Full accuracy score: {accuracy_full}')
    print(confusion_matrix(y, y_pred_full))
    
    first_preds = md.predict(sub_df)
    first_preds_df = sub_df.copy()
    first_preds_df['target'] = first_preds
    
    second_train_df = pd.concat([df_smote,first_preds_df], axis = 0)
    
    print(second_train_df.shape)
    
    x_est = second_train_df.drop('target', axis = 1)
    y_est = second_train_df['target']
    md.fit(x_est,y_est)
    y_pred_est = md.predict(x_est)
    accuracy_est = metrics.accuracy_score(y_est, y_pred_est)
    print(f'Est accuracy score: {accuracy_est}')
    print(confusion_matrix(y_est, y_pred_est))    
    
    sub_pred = md.predict(sub_df)
    submission = pd.DataFrame(test_df['enrollee_id'])
    submission['target'] = sub_pred
    print(submission['target'].value_counts())
    print(submission.shape)
    submission.to_csv(f'{model_label[i]}final.csv', index = False)
    

--------------------------------------------------
xbc
Train ccuracy score: 0.8449165896864642
Test ccuracy score: 0.8409041512714627
[[1921  379]
 [ 353 1948]]
..................................................
Full accuracy score: 0.8477221352808207
[[9643 1859]
 [1644 9858]]
(26836, 159)
Est accuracy score: 0.8651065732598002
[[12256  1894]
 [ 1726 10960]]
0.0    2642
1.0    1190
Name: target, dtype: int64
(3832, 2)
--------------------------------------------------
SVC
Train ccuracy score: 0.9093082649568005
Test ccuracy score: 0.8337317974353401
[[1954  346]
 [ 419 1882]]
..................................................
Full accuracy score: 0.9062771691879673
[[10196  1306]
 [  850 10652]]
(26836, 159)
Est accuracy score: 0.9162691906394396
[[12956  1319]
 [  928 11633]]
0.0    2773
1.0    1059
Name: target, dtype: int64
(3832, 2)
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning]